# Enkapsulacja

**Enkapsulacja** = powiązanie danych z metodami, które na nich działają + kontrola dostępu do danych.

Cel: **Zapobieganie niepoprawnym stanom obiektu.**

## Problem: Niekontrolowany dostęp

In [1]:
class Account:
    def __init__(self, balance):
        self.balance = balance


In [2]:
# client code
a = Account(100)
a.balance = -345  # Niepoprawny stan
a.balance = "asd"  # Niepoprawny typ
print(a.balance)

asd


**Problem**: Możemy ustawić dowolną wartość, nawet niepoprawną.

## Rozwiązanie 1: Gettery i settery

In [3]:
class Account:
    def __init__(self, balance):
        self.balance = balance
    
    def set_balance(self, value):
        if type(value) != int or value < 0:
            print("Podano niepoprawną wartość")
        else:
            self.balance = value
    
    def get_balance(self):
        return self.balance


In [4]:
# client code
a = Account(100)
print(a.get_balance())
a.set_balance(350)
print(a.get_balance())
a.set_balance("asd")  # Odrzucone
print(a.get_balance())

100
350
Podano niepoprawną wartość
350


**Teraz mamy kontrolę**, ale:
- Atrybut `balance` wciąż jest publiczny: `a.balance = -20`
- Dwa sposoby dostępu: `a.balance` i `a.set_balance()` (łamie Zen of Python: "one obvious way")

## Modyfikatory dostępu (Information Hiding)

**Information Hiding** - ukrywanie wewnętrznych danych, aby zapobiec przypadkowym zmianom.

### Pythonowe modyfikatory dostępu:

| Modyfikator | Przykład | Widoczność | Opis |
|-------------|----------|------------|------|
| **Publiczny** | `balance` | Wszędzie | Brak ograniczeń |
| **Chroniony** | `_balance` | Klasa + podklasy | "Wstępujesz na własne ryzyko" |
| **Prywatny** | `__balance` | Tylko klasa | "Wstęp wzbroniony" |

## Rozwiązanie 2: Atrybut prywatny + gettery/settery

In [5]:
class Account:
    def __init__(self, balance):
        self.__balance = balance  # Prywatny atrybut
    
    def get_balance(self):
        return self.__balance
    
    def set_balance(self, value):
        if type(value) != int or value < 0:
            print("Podano niepoprawną wartość")
        else:
            self.__balance = value


In [6]:
# client code
a = Account(150)
print(a.get_balance())
a.set_balance(200)
print(a.get_balance())
# a.__balance  # AttributeError - brak dostępu

150
200


**Lepiej**, ale składnia jest nieczytelna:
```python
a.set_balance(a.get_balance() + 2 * (a.get_balance())**2)
```

vs

```python
a.balance = a.balance + 2 * (a.balance)**2
```

## Rozwiązanie 3: Property (Pythonowy sposób)

In [7]:
class Account:
    def __init__(self, balance):
        self.__balance = balance
    
    def get_balance(self):
        return self.__balance
    
    def set_balance(self, value):
        if type(value) != int or value < 0:
            print("Podano niepoprawną wartość")
        else:
            self.__balance = value
    
    balance = property(get_balance, set_balance)


In [8]:
# client code
a = Account(100)
a.balance = -300  # Wywołuje set_balance
a.balance = 25
print(a.balance)  # Wywołuje get_balance

Podano niepoprawną wartość
25


**Teraz**:
- Czytelna składnia: `a.balance = 25`
- Kontrola dostępu: walidacja w setterze
- Enkapsulacja zachowana: `__balance` jest prywatny

## Property z dekoratorami (idiomatyczny Python)

In [9]:
class Account:
    def __init__(self, balance):
        self.__balance = balance
    
    @property
    def balance(self):
        """Getter"""
        return self.__balance
    
    @balance.setter
    def balance(self, value):
        """Setter z walidacją"""
        if type(value) != int or value < 0:
            print("Podano niepoprawną wartość")
        else:
            self.__balance = value


In [10]:
# client code
a = Account(100)
a.balance = -300  # Odrzucone
a.balance = 25
print(a.balance)

Podano niepoprawną wartość
25


**To jest standard w Pythonie**: dekoratory `@property` i `@balance.setter`.

## Podsumowanie

### Enkapsulacja = Data + Methods + Access Control

**Korzyści**:
- Zapobieganie niepoprawnym stanom obiektu
- Walidacja danych przy ustawianiu
- Ukrycie szczegółów implementacji
- Czytelna składnia (property)

### Pythonowe idiomy:
1. Prywatne atrybuty: `__attribute` (name mangling `_ClassName__attribute`)
2. Property: `@property` + `@attribute.setter`
3. Chronione atrybuty: `_attribute` (konwencja, nie wymuszenie)

### Kiedy stosować?
- Gdy potrzebujesz walidacji
- Gdy atrybut wymaga obliczeń przy odczycie/zapisie
- Gdy chcesz kontrolować dostęp do danych